# PRELIMINARIES

In [1]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
from shapely import geometry
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# LOAD THE DATA

In [2]:
df = pd.read_csv('data/Coffee Brands Footprint.csv',
                index_col=0)
df.head()

,id,brand,name,lat,lng,vicinity
1,ChIJ8aEd9AfyljMRtVgVdqt50kQ,Coffee Bean and Tea Leaf,The Coffee Bean and Tea Leaf,15.164039,120.609505,"Aniceto Gueco St, Angeles"
2,ChIJr_LhvYDtljMRoHrJ5BjoZJE,Coffee Bean and Tea Leaf,The Coffee Bean & Tea Leaf,15.168922,120.580243,"G/F SM City Clark, Manuel A. Roxas Hwy, Clark ..."
3,ChIJc0t13u3tljMRLbLEQRp3vZs,Coffee Bean and Tea Leaf,The Coffee Bean & Tea Leaf,15.169827,120.578192,"Tech Hub, SM Clark, Clark Freeport, Angeles"
4,ChIJ2dePHIjzljMRN2gsqVkO6S4,Coffee Bean and Tea Leaf,The Coffee Bean and Tea Leaf,15.167694,120.564239,"Clark Freeport, Mabalacat"
5,ChIJy19IElXtljMRnnAcNA-x2iA,Starbucks,Starbucks,15.177889,120.530057,"Manuel A. Roxas Hwy, Clark Freeport, Mabalacat"


# INITIATE THE MAP

In [3]:
m = folium.Map(location=[14.577924, 121.073972], tiles="cartodbpositron")
m

In [4]:
marker_cluster = MarkerCluster(control=False).add_to(m)

A necessary step is to create tuple or list of geocode as Folium cannot recognize the geometry object that comes with geopandas or a GeoDataFrame.

In [5]:
#We need this cause apparently, folium can recognize not a geoseries but a normal tuple/list
df.reset_index(drop=True, inplace=True)
df['geocode'] = [[df['lat'][i],df['lng'][i]] for i in range(len(df)) ]

Let us create our markers with a customized tooltip which shows the brand or name of the coffee shop.

In [6]:
for point in df.index: # loop through the plots

        html = """

            <h3>{name}</h3><br>

            """

        popup_contents = folium.Html(html.format(name = df.loc[point, 'name']),
                                             script = True)

        popup = folium.Popup(popup_contents, max_width=1500)

        # mark every addresses in the map from the data
#         folium.Marker(df['geocode'][point],
#                      popup=popup).add_to(marker_cluster)

In [7]:
m

In [8]:
#First Let's Create a Color Dictionary
#Let's Create a Color Dictionary for the Markers
coffee_brands = ["Coffee Bean and Tea Leaf", "Starbucks", "Tim Hortons", "Coffee Project"]

color_dict = {
    "Starbucks": ' #00704A',
    "Coffee Bean and Tea Leaf": '#362d26',
    "Coffee Project": '#654321',
    "Tim Hortons": '#dd0f2d'
}


#Since we need to add a filter for this, we need to add them by brand:
for brand in coffee_brands:
    df_i = df[df['brand']==brand]
    df_i.loc[:, "color"] = color_dict[brand]
    
    
    for index, row in df_i.iterrows():
        
        ####### START OF THE SAME CODE FOR POPUP #######
        html = """

            <h3>{name}</h3><br>

            """

        popup_contents = folium.Html(html.format(name = row['name']),
                                             script = True)

        popup = folium.Popup(popup_contents, max_width=1500)
        ####### END OF THE SAME CODE FOR POPUP #######
        folium.vector_layers.CircleMarker(radius = 8,
                                                  location = (row.lat,
                                                              row.lng),
                                                  color = row.color,
                                                  fill = True,
                                                  fill_color = row.color,
                                                  name = brand,
                                                  popup=popup,
                                                  control = True,
                                                  overlay = True
                                                 ).add_to(marker_cluster)
m

/Users/francisadrianviernes/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/francisadrianviernes/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
m